In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
'''
This function takes the numpy array of CT_Scan and a list of coords from
which voxels are to be cut. The list of voxel arrays are returned. We keep the 
voxels cubic because per pixel distance is same in all directions.
'''
def get_patch_from_list(lung_img, coords, window_size = 10):
    shape = lung_img.shape
    output = []
    lung_img = lung_img + 1024
    for i in range(len(coords)):
        patch =   lung_img[coords[i][0] - 18: coords[i][0] + 18,
                           coords[i][1] - 18: coords[i][1] + 18,
                           coords[i][2] - 18: coords[i][2] + 18]               
        output.append(patch)
    
    return output

'''
Sample a random point from the image and return the coordinates. 
'''
def get_point(shape):
    x = randint(50, shape[2] - 50)
    y = randint(50, shape[1] - 50)
    z = randint(20, shape[0] - 20)
    return np.asarray([z, y, x])

'''
This function reads the training csv file which contains the CT Scan names and
location of each nodule. It cuts many voxels around a nodule and takes random points as 
negative samples. The voxels are dumped using pickle. It is to be noted that the voxels are 
cut from original Scans and not the masked CT Scans generated while creating candidate
regions.
'''
def create_data(path, train_csv_path):
    coords, trainY = [], []
    
    with open(train_csv_path, 'rb') as f:
        lines = f.readlines()
        
        for line in lines:
            row = line.split(',')
            
            if os.path.isfile(path + row[0] + '.mhd') == False:
                continue

            lung_img = sitk.GetArrayFromImage(sitk.ReadImage(path + row[0] + '.mhd'))

            for i in range(-5, 5, 3):
                for j in range(-5, 5, 3):
                    for k in range(-2, 3, 2):
                        coords.append([int(row[3]) + k, int(row[2]) + j, int(row[1]) + i])
                        trainY.append(True)

            for i in range(60):            
                coords.append(get_point(lung_img.shape))
                trainY.append(False)

            trainX = get_patch_from_list(lung_img, coords)

            print (trainX[0].shape, len(trainX))

            pickle.dump(np.asarray(trainX), open('traindata_' + str(counter) + '_Xtrain.p', 'wb'))
            pickle.dump(np.asarray(trainY, dtype = bool),  open('traindata_' + str(counter) + '_Ytrain.p', 'wb'))

            counter = counter + 1
            coords, trainY = [], []

In [3]:
csv_path = PATH['annotations_train']
output_true = PATH['cls_train_20_true']
output_false = PATH['cls_train_20_false']
pred_csv_path = PATH['model_train_pred']
data_path = PATH['src_train']
anno_csv_new = pd.read_csv(csv_path + "annotations_all.csv")
pred_csv_new = pd.read_csv(pred_csv_path + "anno_false_final.csv")

In [4]:
pred_csv_new = pred_csv_new[pred_csv_new.index%12 == 0]

In [5]:
patients=load_train(data_path)
anno_csv_new["file"] = anno_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
anno_csv_new = anno_csv_new.dropna()
pred_csv_new["file"] = pred_csv_new["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
pred_csv_new = pred_csv_new.dropna()

In [7]:
anno_csv_new

,seriesuid,coordX,coordY,coordZ,diameter_mm,file
0,LKDS-00001,-76.449879,-49.540571,229.500000,14.180405,train_subset00/LKDS-00001.mhd
2,LKDS-00003,-82.905787,106.263573,226.224976,5.200858,train_subset00/LKDS-00003.mhd
3,LKDS-00003,-92.705387,191.515214,228.724976,6.723286,train_subset00/LKDS-00003.mhd
4,LKDS-00004,-68.270833,209.412809,167.229885,22.517480,train_subset00/LKDS-00004.mhd
5,LKDS-00005,-147.952967,-164.169479,-273.434241,5.898208,train_subset00/LKDS-00005.mhd
7,LKDS-00007,-36.303583,68.367853,18.875000,6.347767,train_subset00/LKDS-00007.mhd
11,LKDS-00011,79.240951,105.062110,-170.126775,5.838910,train_subset00/LKDS-00011.mhd
12,LKDS-00013,-42.644951,-86.651939,-139.800002,15.884164,train_subset00/LKDS-00013.mhd
14,LKDS-00015,-61.142430,-93.778010,-3.200001,8.262994,train_subset00/LKDS-00015.mhd
15,LKDS-00016,-127.532989,-29.607902,-253.697616,8.269325,train_subset00/LKDS-00016.mhd


In [6]:
create_data(path, train_csv_path)

NameError: name 'path' is not defined